In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from datetime import datetime, timedelta
import pandas as pd

current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-09-08 16:00:00')


In [6]:
from src.inference import load_batch_of_features_from_store
features = load_batch_of_features_from_store(current_date)


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1018005
Connected. Call `.close()` to terminate connection gracefully.
Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1018005
Connected. Call `.close()` to terminate connection gracefully.


Finished: Reading data from Hopsworks, using Hive (65.73s) 


AssertionError: Time-series data is not complete. Make sure your feature pipeline is up and runnning.

In [4]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/878742
Connected. Call `.close()` to terminate connection gracefully.
[LightGBM] [Warning] feature_fraction is set=0.8655813570824598, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8655813570824598
[LightGBM] [Warning] bagging_fraction is set=0.931458948827143, subsample=1.0 will be ignored. Current value: bagging_fraction=0.931458948827143


See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)


In [5]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,0.0,2024-08-09 07:00:00
1,2,0.0,2024-08-09 07:00:00
2,3,0.0,2024-08-09 07:00:00
3,4,3.0,2024-08-09 07:00:00
4,5,0.0,2024-08-09 07:00:00
...,...,...,...
252,261,13.0,2024-08-09 07:00:00
253,262,52.0,2024-08-09 07:00:00
254,263,66.0,2024-08-09 07:00:00
255,264,40.0,2024-08-09 07:00:00


In [6]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/878742
Connected. Call `.close()` to terminate connection gracefully.


In [7]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/878742/fs/874565/fg/1084748


Uploading Dataframe: 0.00% |          | Rows 0/257 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/878742/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x167aa99ff40>, None)